# Sklearn Pipelines Exercise
*Made by viga@itu.dk and thso@itu.dk*

## Introduction

In this exercise you'll be working with the [Wine Quality Dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) from the UCI Machine Learning Repository. The dataset consists of 11 features and a quality score for 4898 white wine samples and 1599 red wine samples. The goal is to predict the quality of the wine based on the features.

The datasets are located in the `data` folder. The `winequality-red.csv` file contains the red wine samples and the `winequality-white.csv` file contains the white wine samples. Lastly, the `winequality.names` file contains a description of the dataset.

The goal of this exercise is to get you familiar with the [Scikit-learn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) API. You'll be using pipelines to perform feature scaling and feature selection.

## Load in the data

You can either load the red-wine dataset or the white-wine dataset. You can also load both datasets and combine them if you want.

Both datasets are available in the `data` folder, and are called `winequality-red.csv` and `winequality-white.csv`.

Hint: You can use the `pd.read_csv()` function to load in the data (remember to check the delimiter!). You can find the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).


In [5]:
import pandas as pd
# load in the data
df_red = pd.read_csv("data/winequality-red.csv", sep=";")
df_white = pd.read_csv("data/winequality-white.csv", sep=";")

# if you want to you can combine the two datasets into one - but this is not necessary
df_red['red'] = 1
df_white['red'] = 0
df = pd.concat([df_red, df_white], ignore_index=True)

# check a few rows of the data - hint: use .head()
display(df_red.head())
display(df_white.head())
display(df.head())


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


## Data Exploration

### Check the number of missing values in the dataset.

Hint: `.isnull()`

Dont worry if there are missing values, we'll handle them later in our pipeline!

In [6]:
df.isnull().sum()

fixed acidity           3
volatile acidity        5
citric acid             8
residual sugar          6
chlorides               7
free sulfur dioxide     6
total sulfur dioxide    6
density                 3
pH                      5
sulphates               6
alcohol                 6
quality                 0
red                     0
dtype: int64

### Check some basic statistics

We want to know the mean, standard deviation, minimum, maximum and quartiles of each feature.
This will give us a good idea of the distribution of the data, and also tell us if we need to do any scaling.

Hint: `.describe()`, If the output is hard to read, you can use `.T` to transpose the dataframe, i.e., swapping the rows and columns.

Do you notice anything strange about the data? Is there anything that stands out to you?

In [8]:
# check some basic statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,6494.0,7.215745,1.296537,3.80000,6.40000,7.00000,7.70000,15.90000
volatile acidity,6492.0,0.339586,0.164520,0.08000,0.23000,0.29000,0.40000,1.58000
citric acid,6489.0,0.318550,0.145211,0.00000,0.25000,0.31000,0.39000,1.66000
residual sugar,6491.0,5.442782,4.758549,0.60000,1.80000,3.00000,8.10000,65.80000
chlorides,6490.0,0.056045,0.035046,0.00900,0.03800,0.04700,0.06500,0.61100
free sulfur dioxide,6491.0,30.529965,17.747755,1.00000,17.00000,29.00000,41.00000,289.00000
total sulfur dioxide,6491.0,115.762209,56.532631,6.00000,77.25000,118.00000,156.00000,440.00000
density,6494.0,0.994696,0.002999,0.98711,0.99234,0.99489,0.99699,1.03898
pH,6492.0,3.218490,0.160781,2.72000,3.11000,3.21000,3.32000,4.01000
sulphates,6491.0,0.531228,0.148832,0.22000,0.43000,0.51000,0.60000,2.00000


We saw that there were some missing values in the dataset, this we can fix in the pipeline, using the [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) from sklearn.

Next we also saw that there was a some differences in the scale of the different variables, so we will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from sklearn to scale the data. This will make it easier for the model to learn the patterns in the data. Especially for the KNN algorithm (which we'll use), which is based on distance, it is important that the data is scaled.

If you think of other transformations that might be useful for this dataset, feel free to try them out!

**Take a look at the [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) module for some inspiration.**

# Splitting the data

Now that we have created our pipeline, we can train the model.

First we need to split the data into a training set and a test set. We will use the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from sklearn to do this. But first we need to split the data into features and labels.

The features are all the columns in the dataset, except for the `quality` column, which are the labels.

We will use the default split of 75% training data, and 25% test data.

Hint: You can use the `random_state` parameter to make sure that the data is split the same way every time you run the code.

The train_test_split function returns four values, the first two are the training and test data, and the last two are the train and test labels.

In [9]:
from sklearn.model_selection import train_test_split
# split the data into X and y
X = df.drop(columns="quality")
y = df["quality"]

# now split the data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4)

# Creating the pipeline

We will now create a pipeline that will handle the missing values and scaling for us, and finally train a KNN model on the data.

We will use the [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class from sklearn to create our pipeline.

The pipeline will consist of three steps, the first step will be to impute the missing values, and the second step will be to scale the data, and the third step will be to train the model.

The pipeline format is a list of tuples, where the first element in the tuple is the name of the step, and the second element is the step itself, e.g.:

```python
pipeline = Pipeline([
	('step_name', step()),
	('step_name', step()),
	('step_name', step()),
])
```

Where the `step_name` is a string, and the `step` is a sklearn object - this can be a "Transformer" object (like `SimpleImputer` and `StandardScaler`) or an "Estimator" object (like `KNeighborsClassifier` or `LinearRegression`).

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# create your pipeline
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")), # fill in missing values
    ("scaler", StandardScaler()),                  # standardize features to mean 0, variance 1
    ("knn", KNeighborsClassifier())                # estimate/classify
])

# Evaluating the model

Now that we have trained the model, we want to evaluate it to see how well it performs.

We will use the [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) function from sklearn to calculate the accuracy of the model.

Since we have created a pipeline, we can simply call the `.fit()` and `.predict()` methods on the pipeline object, and it will handle the preprocessing for us - and importantly in the correct order.

Remember to only call `.fit()` on the training data. Calling `.fit()` on the test data will cause the model to overfit to the test data, and will give you an overly optimistic accuracy score.

* **`.fit(X_train, y_train)` will train the model on the training data.**
* **`.predict(X_train)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_train).**
* **`.predict(X_test)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_test).**

In [11]:
from sklearn.metrics import accuracy_score
import mlflow

# start logging before training/fitting the model
mlflow.autolog()

# fit the pipeline
pipe.fit(X_train, y_train)


# evaluate the pipeline
y_pred = pipe.predict(X_test)
print("Train accuracy: ", accuracy_score(y_train, pipe.predict(X_train)))
print("Test accuracy: ", accuracy_score(y_test, y_pred))



2026/02/06 12:54:32 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2026/02/06 12:54:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/06 12:54:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/06 12:54:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/06 12:54:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/06 12:54:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/06 12:54:33 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/06 12:54:34 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/06 12:54:34 INFO mlflow.store.db.utils: Updating database tables
2026/02/06 12:54:34 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/06 12:54:34 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/06 12:54:34 IN

Train accuracy:  0.6980706075533661
Test accuracy:  0.552
